Chapter 2.a.ii.  Lambda with an indeterminate number of parameters
=======

The previous chapter discussed **Lambda** expressions with a fixed number of parameters.  In this chapter, we will discuss **Lambda** expressions with an unspecified number of parameters.  This is done by using an **ExprRange** of **IndexedVar** expressions in the **ExprTuple** of parameters.  For example $(x_1, ..., x_n) \mapsto (x_1 + ... + x_n)$.  In this new context, we revisit creation of **Lambda** expressions (abstraction), *relabeling* its parameters while retaining its meaning (alpha conversion), and *applying* the **Lambda** function to operators (beta reduction).  As a reminder, the beta reduction rule plays an important role in the **Instantiation** derivation rule, critical for constructing **Prove-It** proofs.

In [1]:
import proveit
%begin lambda_indeterminate_params

## Creating Lambda expressions (abstraction) with an indeterminate number of parameters

We can make a **Lambda** expression with an indeterminate number of parameters by composing parameter entries from an `ExprRange` of `IndexedVar` expressions, such as $x_1, ..., x_n$.  It is important to recognize that although the number of parameters may not be a specific integer value, this number is tied to expressions in a consistent manner.  In the example of $x_1, ..., x_n$, there are $n$ parameters in this range and this same $n$ may be used elsewhere in the expression.  In the following example, we construct a lambda function that takes $2 n$ arguments and produces the dot product between the first $n$ and the last $n$ of these.

In [2]:
from proveit import Lambda, ExprTuple, ExprRange, IndexedVar
from proveit._common_ import a, b, m, n, x, y
from proveit.core_expr_types import Len
from proveit.core_expr_types._common_ import x_1_to_n, y_1_to_n
from proveit.logic import InSet
from proveit.number import one, two, Naturals, Add, Mult

In [3]:
dot_prod_lambda = \
    Lambda((x_1_to_n, y_1_to_n), 
           Add(ExprRange(a, Mult(IndexedVar(x, a),
                                 IndexedVar(y, a)), one, n)))

dot_prod_lambda: (x_{1}, ..x_{_a}.., x_{n}, y_{1}, ..y_{_a}.., y_{n}) -> ((x_{1} * y_{1}) + ..(x_{a} * y_{a}).. + (x_{n} * y_{n}))

It is useful to make the distinction between parameter entries (which may be ExprRange or IndexedVar expressions) and parameter variables (which may only be Variable types).  In the `dot_prod_lambda` example, the parameter entries are $x_1, ..., x_n$ and $y_1, ..., y_n$ but the variables are $x$ and $y$.

In [4]:
dot_prod_lambda.parameters

(x_{1}, ..x_{_a}.., x_{n}, y_{1}, ..y_{_a}.., y_{n})

In [5]:
dot_prod_lambda.parameterVars

(x, y)

Note that parameter variables must be unique (rather than parameter entries).

In [6]:
from proveit import ParameterCollisionError
try:
    Lambda((x_1_to_n, x), Mult(x_1_to_n))
    assert False, "Expected an ParameterCollisionError error."
except ParameterCollisionError as e:
    print("Expected error:", e)

Expected error: Parameter variables must be unique.  (x_{1}, ..x_{_a}.., x_{n}, x) does not satisfy this criterion.


In [7]:
try:
    Lambda((IndexedVar(x, n), IndexedVar(x, m)), Mult(IndexedVar(x, n), IndexedVar(x, m)))
    assert False, "Expected an ParameterCollisionError error."
except ParameterCollisionError as e:
    print("Expected error:", e)

Expected error: Parameter variables must be unique.  (x_{n}, x_{m}) does not satisfy this criterion.


Also, none of the parameter variables may occur as free variables within any parameter index.

In [8]:
try:
    Lambda((IndexedVar(x, n), IndexedVar(y, x)), Mult(IndexedVar(x, n), IndexedVar(y, x)))
    assert False, "Expected an ParameterCollisionError error."
except ParameterCollisionError as e:
    print("Expected error:", e)

Expected error: Parameter variables may not occur as free variables in parameter indices.  (x_{n}, y_{x}) does not satisfy this criterion.


Parameters are not restricted to occurrences where the ranges matches.  Any ambiguity must be resolved when the **Lambda** expression is applied, however.  Also, relabeling will not be allowed as we shall see in the next section.

In [9]:
inconsistent_dot_prod = Lambda((x_1_to_n, y_1_to_n), Add(ExprRange(a, Mult(IndexedVar(x, a),
                                                                           IndexedVar(y, a)), one, m)))

inconsistent_dot_prod: (x_{1}, ..x_{_a}.., x_{n}, y_{1}, ..y_{_a}.., y_{n}) -> ((x_{1} * y_{1}) + ..(x_{a} * y_{a}).. + (x_{m} * y_{m}))

## Relabeling with an indeterminant number of parameters (alpha conversion)

When a parameter variable occurs in the **Lambda** body only within ranges that match the range of the parameter entry, relabeling is straightforward.  For example, with

In [10]:
dot_prod_lambda

(x_{1}, ..x_{_a}.., x_{n}, y_{1}, ..y_{_a}.., y_{n}) -> ((x_{1} * y_{1}) + ..(x_{a} * y_{a}).. + (x_{n} * y_{n}))

both $x$ and $y$ have consistent $1$ to $n$ ranges in the body and the parameters.  So we can relabel with

In [11]:
dot_prod_lambda_relabeled = dot_prod_lambda.relabeled({x:a, y:b})

dot_prod_lambda_relabeled: (a_{1}, ..a_{_a}.., a_{n}, b_{1}, ..b_{_a}.., b_{n}) -> ((a_{1} * b_{1}) + ..(a_{_a} * b_{_a}).. + (a_{n} * b_{n}))

In [12]:
assert dot_prod_lambda_relabeled == dot_prod_lambda

When they do not match, a `DisallowedParameterRelabeling` exception is raised whenever relabeling is attempted.  The attempt may be direct:

In [13]:
from proveit import DisallowedParameterRelabeling
try:
    inconsistent_dot_prod.relabeled({x:a, y:b})
    assert False, "Expected an DisallowedParameterRelabeling error."
except DisallowedParameterRelabeling as e:
    print("Expected error:", e)

Expected error: Cannot relabel x in (x_{1}, ..x_{_a}.., x_{n}, y_{1}, ..y_{_a}.., y_{n}) -> ((x_{1} * y_{1}) + ..(x_{a} * y_{a}).. + (x_{m} * y_{m})); relabeling is only allowed when all occurrences of a range of parameters matches the exact range appearing as parameters (otherwise, the bound verses free portions of the range may be ambiguous).


Or relabeling may be indirect when the system tries to automaticically avoid a variable collision:

In [14]:
try_this = Lambda(b, Lambda((x_1_to_n, y_1_to_n), Add(ExprRange(a, Mult(b, IndexedVar(x, a),
                                                                        IndexedVar(y, a)), one, m))))

try_this: b -> [(x_{1}, ..x_{_a}.., x_{n}, y_{1}, ..y_{_a}.., y_{n}) -> ((b * x_{1} * y_{1}) + ..(b * x_{a} * y_{a}).. + (b * x_{m} * y_{m}))]

In [15]:
try:
    try_this.apply(x) # SHOULD NOT BE ALLOWED!
    assert False, "Expected an DisallowedParameterRelabeling error."
except DisallowedParameterRelabeling as e:
    print("Expected error:", e)

Expected error: Cannot relabel x in (x_{1}, ..x_{_a}.., x_{n}, y_{1}, ..y_{_a}.., y_{n}) -> ((b * x_{1} * y_{1}) + ..(b * x_{a} * y_{a}).. + (b * x_{m} * y_{m})); relabeling is only allowed when all occurrences of a range of parameters matches the exact range appearing as parameters (otherwise, the bound verses free portions of the range may be ambiguous). Thus, a collision of variable names induced by the following replacement map could not be avoided: {b: x}.


It is, however, possible to perform an internal relabeling in such cases in the process of performing an lambda application.  We will show an example of this in the next section.

## Application with an indeterminant number of parameters (beta reduction)

Starting with a simple case, let us call the `apply` method on our `dot_prod_labmda` example to perform beta reduction.

In [16]:
dot_prod_lambda

(x_{1}, ..x_{_a}.., x_{n}, y_{1}, ..y_{_a}.., y_{n}) -> ((x_{1} * y_{1}) + ..(x_{a} * y_{a}).. + (x_{n} * y_{n}))

Let's apply this to some operands for the simple case where they are composed of two expression ranges that have the same start and end indices as the corresponding parameters.

In [17]:
operands = [ExprRange(a, a, one, n), ExprRange(a, Add(a, a), one, n)]

operands: (1, ..a.., n, 1 + 1, ..a + a.., n + n)

In [18]:
requirements = []
dot_prod_lambda.apply(*operands, assumptions=[InSet(n, Naturals)],
                     requirements=requirements)

(1 * (1 + 1)) + ..(a * (a + a)).. + (n * (n + n))

Note that this time we supplied assumptions, specifically that $n \in \mathbb{N}$, and made a list to pass back requirements.  In order to make this step, we need to prove that the lengths of the operands match lengths of corresponding parameters.  In order to prove it in this case, we need to know that $n$ is in the set of natural numbers (otherwise, this lambda expression does not make sense and should not apply to anything anyways).  The requirements that are passed back are precisely the known truths that prove that the lengths match as is necessary in order to assure that this beta reduction is valid.  This will get used when beta reduction is employed to perform the **instantiation** derivation rule.

In [19]:
requirements

We can also perform `apply` such that multiple operand entries correspond to a given parameter entry as long as we can meet the length-matching requirements under the provided assumptions.  In such cases, expression ranges containing these parameters will expand to accommodate the different operand entries.  Also, it doesn't matter what the start and end indices are, as long as we meet the length matching requirements.  Consider the following operands for an application of `dot_prod_lambda`.

In [20]:
from proveit._common_ import k
from proveit.logic import Equals
from proveit.number import zero
operands = [ExprRange(a, Mult(a, a), zero, k), x,
            ExprRange(a, a, one, m), 
            ExprRange(a, Add(a, a), zero, k), y,
            ExprRange(a, a, one, m)]

operands: (0 * 0, ..a * a.., k * k, x, 1, ..a.., m, 0 + 0, ..a + a.., k + k, y, 1, ..a.., m)

In [21]:
assumption1 = Equals(Len(operands[:3]),
                     Len(ExprRange(a, a, one, n)))

assumption1: |(0 * 0, ..a * a.., k * k, x, 1, ..a.., m)| = |(1, ..a.., n)|

In [22]:
assumption2 = Equals(Len(operands[3:]),
                     Len(ExprRange(a, a, one, n)))

assumption2: |(0 + 0, ..a + a.., k + k, y, 1, ..a.., m)| = |(1, ..a.., n)|

In [23]:
requirements = []
dot_prod_lambda.apply(*operands, assumptions=[InSet(n, Naturals),
                                             assumption1,
                                             assumption2],
                     requirements=requirements)

((0 * 0) * (0 + 0)) + ..((a * a) * (a + a)).. + ((k * k) * (k + k)) + (x * y) + (1 * 1) + ..(a * a).. + (m * m)

In [24]:
requirements

To ensure unambiguous and straightforward behavior when different parameter ranges are involved in the same expression range (as in the dot product case), their operands must be in exact correspondence with respect to range start and end indices (and whether or not an entry is a range).  Otherwise, a `LambdaApplicationError` exception will be raised as in the following demonstration.  In this example, `assumption1` is used again so there is a proper alignment of lengths with $n$ elements for $x$ and $n$ elements for $y$.  However, the internal structures of $x$ and $y$ are not aligned to each other.

In [25]:
from proveit._common_ import k
new_operands = [*operands[:3], 
                ExprRange(a, Add(a, a), one, n)]

new_operands: (0 * 0, ..a * a.., k * k, x, 1, ..a.., m, 1 + 1, ..a + a.., n + n)

In [26]:
from proveit import LambdaApplicationError
try:
    dot_prod_lambda.apply(*new_operands, 
                          assumptions=[InSet(n, Naturals), 
                                       assumption1],
                         requirements=requirements)
    assert False, "Expecting LambdaApplicationError"
except LambdaApplicationError as e:
    print("Expected error:", e)

Expected error: Failure to apply (x_{1}, ..x_{_a}.., x_{n}, y_{1}, ..y_{_a}.., y_{n}) -> ((x_{1} * y_{1}) + ..(x_{a} * y_{a}).. + (x_{n} * y_{n})) to ((0 * 0), ..(a * a).., (k * k), x, 1, ..a.., m, (1 + 1), ..(a + a).., (n + n)) assuming (n in Naturals, |(0 * 0, ..a * a.., k * k, x, 1, ..a.., m)| = |(1, ..a.., n)|):
Improper replacement: Improper replacement of (x_{1} * y_{1}), ..(x_{a} * y_{a}).., (x_{n} * y_{n}) via {(x_{1}, ..x_{_a}.., x_{n}): (0 * 0, ..a * a.., k * k, x, 1, ..a.., m), (y_{1}, ..y_{_a}.., y_{n}): (1 + 1, ..a + a.., n + n), x: {(x_{1}, ..x_{_a}.., x_{n})}, y: {(y_{1}, ..y_{_a}.., y_{n})}}:
When expanding IndexedVar's within an ExprRange whose parameter is the index, their expansion ExprRange indices must all match. ((1 + 1), ..(a + a).., (n + n),) vs ((0 * 0), ..(a * a).., (k * k), x, 1, ..a.., m) do not match as respected expansions for y_{a} and x_{a}. 


Of course, when parameter ranges are not involved in the same expression range, this restriction does not apply.

In [27]:
add2n_lambda = Lambda((x_1_to_n, y_1_to_n), 
                       Add(x_1_to_n, y_1_to_n))

add2n_lambda: (x_{1}, ..x_{_a}.., x_{n}, y_{1}, ..y_{_a}.., y_{n}) -> (x_{1} + ..x_{_a}.. + x_{n} + y_{1} + ..y_{_a}.. + y_{n})

In [28]:
requirements = []
add2n_lambda.apply(*new_operands,
                   assumptions=[InSet(n, Naturals), assumption1],
                   requirements=requirements)

(0 * 0) + ..(_a * _a).. + (k * k) + x + 1 + .._a.. + m + (1 + 1) + ..(_a + _a).. + (n + n)

Furthermore, to ensure unambiguous and straightforward lambda application, we do not allow a single operand entry to correspond with multiple parameter entries or crossing boundaries in any way.

In [29]:
operands = [ExprRange(a, Mult(a, a), one, Mult(two, n))]

operands: (1 * 1, ..a * a.., (2 * n) * (2 * n))

In [30]:
assumption = Equals(Len(operands),
                    Len([ExprRange(a, a, one, n), 
                         ExprRange(a, a, one, n)]))

assumption: |(1 * 1, ..a * a.., (2 * n) * (2 * n))| = |(1, ..a.., n, 1, ..a.., n)|

In [31]:
try:
    add2n_lambda.apply(*operands, assumptions=[assumption],
                       requirements=requirements)
    assert False, "Expecting LambdaApplicationError"
except LambdaApplicationError as e:
    print("Expected error:", e)

Expected error: Failure to apply (x_{1}, ..x_{_a}.., x_{n}, y_{1}, ..y_{_a}.., y_{n}) -> (x_{1} + ..x_{_a}.. + x_{n} + y_{1} + ..y_{_a}.. + y_{n}) to ((1 * 1), ..(a * a).., ((2 * n) * (2 * n)),) assuming (|(1 * 1, ..a * a.., (2 * n) * (2 * n))| = |(1, ..a.., n, 1, ..a.., n)|,):
Parameter/argument length mismatch or unproven length equality for correspondence with x_{1}, ..x_{_a}.., x_{n}.


### Providing alternative expansions

A variable may occur in an expression in various forms, indexed over different ranges.  In order to treat the various forms that a range of parameters may occur in an unambiguous and versatile manner, you may pass an optional `equiv_alt_expansions` dictionary to the `apply` method for specifying various expansions for the different alternative forms.  The rule in doing this is fairly simple and straightforward, but allows for a lot of versatility.  Basically, if $x_i, ..., x_j$ is a range of parameters of the lambda expression, an `equiv_alt_expansions` may have various keys that are alternative ways of representing $(x_i, ..., x_j)$, such as $(x_i, x_{i+1}, ..., x_{j-1}, x_j)$ assuming $j-i \geq 1$, and the corresponding values of the dictionary are alternative ways of representing the **ExprTuple** of operands supplied for the $x_i, ..., x_j$ parameters.  These alternative expansions can provide the information needed to expand the variable in its various forms.  The requirements to allow for these alternative expansions is straightforward.  The alternative ways of representing $(x_i, ..., x_j)$ must be equal to $(x_i, ..., x_j)$ and the corresponding values of the `equiv_alt_expansions` dictionary must be equal to the **ExprTuple** of operands supplied for $x_i, ..., x_j$.

The following is an example that demonstrates the versatility of this feature and even includes some partial masking of a range of parameters to make it interesting.

In [32]:
from proveit import varRange
from proveit._common_ import A, B, C, D, i, j, k, m
from proveit.core_expr_types._common_ import A_1_to_m, A_i_to_j
from proveit.logic import Not, And, Or, Forall
from proveit.number import one, Neg, subtract, NaturalsPos
A_1_to_j, A_m = varRange(A, one, j), IndexedVar(A, m)

A_1_to_j: A_{1}, ..A_{_a}.., A_{j}
A_m: A_{m}

In [33]:
partially_masked_lambda = Lambda(A_1_to_m, And(A_1_to_j, Forall(A_i_to_j, Or(A_i_to_j)), A_m))

partially_masked_lambda: (A_{1}, ..A_{_a}.., A_{m}) -> (A_{1} and ..A_{_a}.. and A_{j} and [forall_{A_{i}, ..A_{_a}.., A_{j}} (A_{i} or ..A_{_a}.. or A_{j})] and A_{m})

On its own, this **Lambda** expression is ambiguous.  There are different interpretations depending upon the order of the $1$, $i$, $j$, and $m$ indices.  The `assumptions` and the `equiv_alt_expansions` supplied when calling the `apply` must resolve any ambiguity.  So let's set this up for some unambiguous interpretation when we apply this lambda expression to some `operands`.

In [34]:
operands = ExprTuple(ExprRange(k, Not(IndexedVar(B, k)), one, subtract(i, one)), 
                    varRange(C, one, i),
                    Or(A, D))

operands: ([not](B_{1}), ..[not](B_{k}).., [not](B_{i - 1}), C_{1}, ..C_{_a}.., C_{i}, A or D)

This gives us the replacement that should be used when encountering the range $A_1, ..., A_m$, but we really need to know what to do when we encounter $A_1, ..., A_j$, $A_i, ..., A_j$, and $A_m$.  First, we will need some assumptions for our arbitrary (for demonstration purposes) scenario.

In [35]:
assumptions = (Equals(m, Add(j, one)), InSet(subtract(m, one), Naturals), InSet(i, NaturalsPos), 
               InSet(j, NaturalsPos), #InSet(Add(j, Neg(i), one), Naturals),
               Equals(j, subtract(Mult(two, i), one)),
               Equals(Add(j, Neg(i), one), i),
               Equals(Add(j, one), Add(i, i)))

assumptions: (m = (j + 1), (m - 1) in Naturals, i in NaturalsPos, j in NaturalsPos, j = ((2 * i) - 1), (j - i + 1) = i, (j + 1) = (i + i))

The last two assumptions are redundant so we don't have to bother proving them for this demonstration.  They were chosen to give us precisely what we need for proving the requirements needed for our lambda application demonstration.  Now let's make some needed alternatives to $(A_1, ..., A_m)$ and prove they are equivalent to it by calling the `ExprTuple.merger` method which will automatically apply some theorems.

In [36]:
alt_A_form1 = ExprTuple(varRange(A, one, subtract(i, one)), A_i_to_j, A_m)

alt_A_form1: (A_{1}, ..A_{_a}.., A_{i - 1}, A_{i}, ..A_{_a}.., A_{j}, A_{m})

In [37]:
# And maybe we'll have a simpler way to do this:
assumptions[5].subLeftSideInto(assumptions[2].prove(assumptions), assumptions)

{i in NaturalsPos, (j - i + 1) = i} |- (j - i + 1) in NaturalsPos

In [38]:
alt_A_form1.merger(assumptions)

{j in NaturalsPos, i in NaturalsPos, (j - i + 1) = i, (m - 1) in Naturals, m = (j + 1)} |- (A_{1}, ..A_{_a}.., A_{i - 1}, A_{i}, ..A_{_a}.., A_{j}, A_{m}) = (A_{1}, ..A_{_a}.., A_{m})

Here we employ a trick using an `InnerExpr` object.  We'll discuss how that works in a later chapter.

In [39]:
alt_A_form2 = alt_A_form1.innerExpr(assumptions)[:2].merged(assumptions=assumptions)

alt_A_form2: (A_{1}, ..A_{_a}.., A_{j}, A_{m})

In [40]:
from proveit import extract_var_tuple_indices
extract_var_tuple_indices(alt_A_form1).merger(assumptions)

{j in NaturalsPos, i in NaturalsPos, (j - i + 1) = i, (m - 1) in Naturals, m = (j + 1)} |- (1, .._a.., i - 1, i, .._a.., j, m) = (1, .._a.., m)

Now we are ready to demonstrate this lambda application using multiple alternative expansions via `equiv_alt_expansions`.

In [41]:
m_eq = assumptions[4].subRightSideInto(assumptions[0], assumptions)

m_eq: {m = (j + 1), j = ((2 * i) - 1)} |- m = (((2 * i) - 1) + 1)

In [42]:
m_eq.innerExpr().rhs.simplify(assumptions)

{m = (j + 1), j = ((2 * i) - 1), i in NaturalsPos} |- m = (2 * i)

In [43]:
requirements = []
partially_masked_lambda.apply(*operands, assumptions=assumptions, requirements=requirements,
                              equiv_alt_expansions={alt_A_form1:operands, alt_A_form2:operands})

[not](B_{1}) and ..[not](B_{_a}).. and [not](B_{i + (-1)}) and C_{1} and ..C_{_a}.. and C_{i} and [forall_{A_{i}, ..A_{_a}.., A_{j}} (A_{i} or ..A_{_a}.. or A_{j})] and (A or D)

And these were the requirements to make that happen, making sure that lengths match properly as well as certain index ranges.

In [44]:
requirements

Notice how the inner $\forall_{A_i, .., A_j}$ masks a certain range of the $A$ variables and remains unchanged.  If we turn on the `allow_relabeling` flag, we can propagate changes as long as the replacement yields proper parameters.  In this case, it does:

In [45]:
partially_masked_lambda.apply(*operands, assumptions=assumptions,
                              equiv_alt_expansions={alt_A_form1:operands, alt_A_form2:operands},
                             allow_relabeling=True)

[not](B_{1}) and ..[not](B_{_a}).. and [not](B_{i + (-1)}) and C_{1} and ..C_{_a}.. and C_{i} and [forall_{C_{1}, ..C_{_a}.., C_{i}} (C_{1} or ..C_{_a}.. or C_{i})] and (A or D)

But of we change our `operands` such that it does not provide proper relabeling, then no relabeling will occur either way (whether `allow_relabeling` on or off).

In [46]:
operands = ExprTuple(ExprRange(k, Not(IndexedVar(B, k)), one, subtract(i, one)), 
                     ExprRange(k, Not(IndexedVar(C, k)), one, i), Or(A, D))

operands: ([not](B_{1}), ..[not](B_{k}).., [not](B_{i - 1}), [not](C_{1}), ..[not](C_{k}).., [not](C_{i}), A or D)

In [47]:
partially_masked_lambda.apply(*operands, assumptions=assumptions,
                              equiv_alt_expansions={alt_A_form1:operands, alt_A_form2:operands})

[not](B_{1}) and ..[not](B_{_a}).. and [not](B_{i + (-1)}) and [not](C_{1}) and ..[not](C_{_a}).. and [not](C_{i}) and [forall_{A_{i}, ..A_{_a}.., A_{j}} (A_{i} or ..A_{_a}.. or A_{j})] and (A or D)

In [48]:
partially_masked_lambda.apply(*operands, assumptions=assumptions,
                              equiv_alt_expansions={alt_A_form1:operands, alt_A_form2:operands},
                             allow_relabeling=True)

[not](B_{1}) and ..[not](B_{_a}).. and [not](B_{i + (-1)}) and [not](C_{1}) and ..[not](C_{_a}).. and [not](C_{i}) and [forall_{A_{i}, ..A_{_a}.., A_{j}} (A_{i} or ..A_{_a}.. or A_{j})] and (A or D)

We will discuss these features and its limitations in more detail in a later chapter (ExprRange/IndexedVar expansions in ExprTuples).  Here we demonstrated some of the versatility of using `equiv_alt_expansions`.

In [49]:
%end lambda_indeterminate_params

# Next chapter: <a href="tutorial04_relabeling.ipynb">ToDo</a>

## <a href="tutorial00_introduction.ipynb#contents">Table of Contents</a>